In [ ]:
vlib = c("tidyverse", "data.table", "Seurat", "tidyseurat", "SeuratDisk",
         "ggsci", "ggpubr", "SCopeLoomR")
lapply(vlib, require, character.only = TRUE, quietly = TRUE) |> suppressMessages()
setwd("202111-CHIP/")

In [ ]:

seurat_obj = readRDS("CHIP_scRNA/hdWGCNA/230808_hdWGCNA.RDS")

update_md = fread("/data/podo/Projects/project_HS/202111-CHIP/CHIP_scRNA/230724_CHIP_subset/final_anno_230802/230809_all_merged.txt.gz")
colnames(update_md) = c("cell", "rn_anno_l1", "rn_anno_l2")

temp_md = seurat_obj@meta.data %>% rownames_to_column("cell") 

new_md = left_join(temp_md, update_md) %>%
  mutate(anno_l1 = ifelse(is.na(rn_anno_l1) ==FALSE, rn_anno_l1, anno_l1)) %>%
  mutate(anno_l2 = ifelse(is.na(rn_anno_l2) ==FALSE, rn_anno_l2, anno_l2)) %>%
  select(-rn_anno_l1, -rn_anno_l2) %>%
  mutate(VAF_bin = ifelse(CHIP_MaxAF >= 0.1, "VAF>=0.1",
                          ifelse(CHIP_MaxAF >= 0.02, "0.1>VAF>=0.02", "Negative"))) %>%
  mutate(VAF_bin= factor(VAF_bin, levels = c("VAF>=0.1", "0.1>VAF>=0.02", "Negative"))) %>%
  mutate(time = factor(time, levels = c("base", "1st"))) %>%
  mutate(group = paste(sep = "-", VAF_bin, anno_l2, time, Pathology)) %>%
  column_to_rownames("cell")

seurat_obj@meta.data = new_md
seurat_obj@meta.data = seurat_obj@meta.data %>% 
  mutate(DNMT3A = ifelse(CHIP_ID %in% list_targetGene_sampleID$DNMT3A, list_targetGene_levels$DNMT3A[1], list_targetGene_levels$DNMT3A[2]),
         TET2 = ifelse(CHIP_ID %in% list_targetGene_sampleID$TET2, list_targetGene_levels$TET2[1], list_targetGene_levels$TET2[2]),
         PPM1D = ifelse(CHIP_ID %in% list_targetGene_sampleID$PPM1D, list_targetGene_levels$PPM1D[1], list_targetGene_levels$PPM1D[2])) %>%
  mutate(DNMT3A = factor(DNMT3A, levels = list_targetGene_levels$DNMT3A),
         TET2 = factor(TET2, levels = list_targetGene_levels$TET2),
         PPM1D = factor(PPM1D, levels = list_targetGene_levels$PPM1D))

